# Plotly Interactive Visualization Demo

This notebook demonstrates the use of the plotting utilities provided in the `plotting_utils.py` module.

## Setup

First, let's import the necessary libraries and the plotting utility function.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import the plotting utility function
from plotting_utils import create_box_violin_plots

## Create Sample Data

Let's create a sample DataFrame with the expected columns: 'eventType' and 'reconstruction_error'.

In [2]:
# Set random seed for reproducibility
np.random.seed(42)

# Create sample data
event_types = ['Normal', 'Anomaly', 'Warning']
data = []

# Generate data for each event type with different distributions
for event_type in event_types:
    if event_type == 'Normal':
        # Normal events have lower reconstruction errors
        errors = np.random.normal(loc=0.5, scale=0.2, size=100)
    elif event_type == 'Anomaly':
        # Anomalies have higher reconstruction errors
        errors = np.random.normal(loc=2.0, scale=0.5, size=30)
    else:  # Warning
        # Warnings have intermediate reconstruction errors
        errors = np.random.normal(loc=1.2, scale=0.3, size=50)
    
    # Ensure all errors are positive
    errors = np.abs(errors)
    
    # Add to data list
    for error in errors:
        data.append({'eventType': event_type, 'reconstruction_error': error})

# Create DataFrame
df = pd.DataFrame(data)

# Display the first few rows
df.head()

,eventType,reconstruction_error
0,Normal,0.599343
1,Normal,0.472347
2,Normal,0.629538
3,Normal,0.804606
4,Normal,0.453169


## Explore the Data

Let's look at some basic statistics of our sample data.

In [3]:
# Count of each event type
df['eventType'].value_counts()

eventType
Normal     100
Warning     50
Anomaly     30
Name: count, dtype: int64

In [4]:
# Summary statistics for reconstruction_error by event type
df.groupby('eventType')['reconstruction_error'].describe()

,count,mean,std,min,25%,50%,75%,max
eventType,,,,,,,,
Anomaly,30.0,2.026393,0.524573,1.040614,1.724704,1.984694,2.270578,3.231621
Normal,100.0,0.479710,0.180352,0.023949,0.379819,0.474609,0.581190,0.870456
Warning,50.0,1.228894,0.292217,0.717755,0.967238,1.272412,1.425152,2.016051


## Create Box and Violin Plots

Now, let's use the `create_box_violin_plots` function to visualize the distribution of reconstruction errors for each event type.

In [5]:
# Create plots for 'Normal' events
fig_normal = create_box_violin_plots(df, 'Normal')
fig_normal.show()

In [8]:
# Create plots for 'Anomaly' events
fig_anomaly = create_box_violin_plots(df, 'Anomaly')
fig_anomaly.show()

In [9]:
# Create plots for 'Warning' events
fig_warning = create_box_violin_plots(df, 'Warning')
fig_warning.show()

## Create a Custom Visualization

Let's create a custom visualization that compares all event types in a single figure.

In [6]:
# Create a figure with subplots
fig = make_subplots(rows=1, cols=2,
                   subplot_titles=('Box Plot Comparison', 'Violin Plot Comparison'),
                   shared_yaxes=True)

# Colors for different event types
colors = {'Normal': 'blue', 'Anomaly': 'red', 'Warning': 'orange'}

# Add box plots for each event type
for event_type in event_types:
    filtered_df = df[df['eventType'] == event_type]
    
    # Add box plot
    fig.add_trace(
        go.Box(y=filtered_df['reconstruction_error'],
               name=event_type,
               boxpoints='all',
               jitter=0.3,
               pointpos=-1.8,
               marker_color=colors[event_type]),
        row=1, col=1
    )
    
    # Add violin plot
    fig.add_trace(
        go.Violin(y=filtered_df['reconstruction_error'],
                 name=event_type,
                 box_visible=True,
                 meanline_visible=True,
                 line_color=colors[event_type]),
        row=1, col=2
    )

# Update layout
fig.update_layout(
    title='Comparison of Reconstruction Error Distributions by Event Type',
    height=600,
    width=1200,
    template='plotly_white'
)

# Update y-axis labels
fig.update_yaxes(title_text='Reconstruction Error', row=1, col=1)
fig.update_yaxes(title_text='Reconstruction Error', row=1, col=2)

# Show the figure
fig.show()

## Interactive Features

Let's create an interactive widget to select which event type to display.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create a dropdown widget for event type selection
event_dropdown = widgets.Dropdown(
    options=event_types,
    value='Normal',
    description='Event Type:',
    disabled=False,
)

# Create an output widget to display the plot
output = widgets.Output()

# Define a function to update the plot based on the selected event type
def update_plot(change):
    with output:
        clear_output(wait=True)
        selected_event = change['new']
        fig = create_box_violin_plots(df, selected_event)
        fig.show()

# Register the callback function
event_dropdown.observe(update_plot, names='value')

# Display the widget and initial plot
display(event_dropdown)
display(output)

# Show the initial plot
with output:
    fig = create_box_violin_plots(df, 'Normal')
    fig.show()

Dropdown(description='Event Type:', options=('Normal', 'Anomaly', 'Warning'), value='Normal')

Output()

## Conclusion

In this notebook, we've demonstrated how to use the `create_box_violin_plots` function from the `plotting_utils` module to visualize the distribution of reconstruction errors for different event types. We've also created a custom visualization that compares all event types in a single figure and added an interactive widget to select which event type to display.

The box and violin plots provide complementary views of the data distribution:
- Box plots show the median, quartiles, and outliers
- Violin plots show the full distribution shape

These visualizations help us understand the differences in reconstruction error distributions between normal events, anomalies, and warnings.